In [2]:
import numpy as np

In [3]:
import sys;
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, neg, pairing, Z1, Z2
import galois

GF = galois.GF(curve_order)

In [8]:
print("\n======== Affichage de nos polynomes ========")

# Prouver que u*v = w+ht
u = galois.Poly([14774563938491510775016323878048660684770145970280823181996287825938670734546, 3648040478639879203707734290876212514758060733402672390616367364429301415868, 10032111316259667810196269299909584415584667016857349074195010252180578894213, 7296080957279758407415468581752425029516121466805344781232734728858602830837, 8025689053007734248157015439927667532467733613485879259356008201744463116328, 21888242871839275222246405745257275088548364400416034343698204186575808495064], field=GF)
v = galois.Poly([20429026680383323540763312028906790082645140107054965387451657240804087929235, 14592161914559516814830937163504850059032242933610689562465469457717205663881, 7296080957279758407415468581752425029516121466805344781232734728858602830952, 7296080957279758407415468581752425029516121466805344781232734728858602834704, 16051378106015468496314030879855335064935467226971758518712016403488926226275, 1809], field=GF)
w = galois.Poly([11126523459851631571308589587172448170012085236878150791379920461509369318592, 3648040478639879203707734290876212514758060733402672390616367364429301416191, 13680151794899547013904003590785796930342727750260021464811377616609880307969, 7296080957279758407415468581752425029516121466805344781232734728858602837524, 8025689053007734248157015439927667532467733613485879259356008201744463107336, 3647], field=GF)
t = galois.Poly([1, 21888242871839275222246405745257275088548364400416034343698204186575808495596, 175, 21888242871839275222246405745257275088548364400416034343698204186575808494882, 1624, 21888242871839275222246405745257275088548364400416034343698204186575808493853, 720], field=GF)
h = galois.Poly([0, 18568526036276985146872367540559921700118529133019602468237309884945144207081, 8414813370729321363219173764287796867375260091715497647688420720616921933174, 2760350628837508597472185613429667469500265954941355442233051305751504736345, 1240333762737558929260629658897912255017740649356908612809564903905962484152, 19213013187503363806194067265281385911059119862587407923912868119327654122536], field=GF)

# check initial
h_quo = (u * v - w) // t
h_rem = (u * v - w) % t

print("u = ", u)
print("v = ", v)
print("w = ", w)
print("t = ", t)
print("h = ", h)
print("  -> h_quo = ", h_quo)
print("  -> h_rem = ", h_rem)

if (h_rem != 0):
    print("\n!! Les équations sont cassées !! ")
    sys.exit()

print("\n => Les équations sont bonnes")


nombre_var=len(u)

# evaluate at 8 - Normalement c'est le trusted ici
tau = GF(123)

print("tau =  ", tau)

tau_G1=[]
for i in range(0, nombre_var):
    tau_G1.append(multiply(G1, int(tau**i)))
tau_G2=[]
for i in range(0, nombre_var):
    tau_G2.append(multiply(G2, int(tau**i)))

tau_t_G1=[]
t_tau=int(t(tau))
for i in range(0, nombre_var):
    tau_t_G1.append(multiply(multiply(G1, t_tau),int(tau**i)))

#tau_G2=[tauG2_0,tauG2_1,tauG2_2,tauG2_3,tauG2_4,tauG2_5,tauG2_6,tauG2_7,tauG2_8,tauG2_9,tauG2_10] #<[tau^0*G2]2,[tau^1*G2]2,[tau^2*G2]2,...[]>
print("\n======== Calcul de tau^i*G1 , tau^i*G2 et tau^i_t_G1========")
print("[tau_G1]1 = ", tau_G1)
print("[tau_G2]2 = ", tau_G2)
print("[tau_t_G1]1 = ", tau_t_G1)


print("\n======== Calcul de [A],[B] et [C] ========")
# CALCULE DE encodeCoeffU noté A
A=0
X_U=[]
for i in range(0, nombre_var):
    X_U.append(multiply(tau_G2[i], int(u.coeffs[::-1][i])))
encodeCoeffU=Z1
for i in range(0, nombre_var):
    encodeCoeffU=add(encodeCoeffU, X_U[i])
A=encodeCoeffU    
print("[A]2= ",A)


# CALCULE DE encodeCoeffV noté B
B=0
X_V=[]
for i in range(0, nombre_var):
    X_V.append(multiply(tau_G1[i], int(v.coeffs[::-1][i])))
encodeCoeffV=Z1
for i in range(0, nombre_var):
    encodeCoeffV=add(encodeCoeffV, X_V[i])
B=encodeCoeffV
print("[B]1= ",B)

# CALCULE DE encodeCoeffW
X_W=[]
for i in range(0, nombre_var):
    X_W.append(multiply(tau_G1[i], int(w.coeffs[::-1][i])))
encodeCoeffW=Z2
for i in range(0, nombre_var):
    encodeCoeffW=add(encodeCoeffW, X_W[i])
print("[W]1=",encodeCoeffW)

# CALCULE DE encodeCoeffHT
X_HT=[]
for i in range(0, len(h.coeffs)):
    X_HT.append(multiply(tau_t_G1[i], int(h.coeffs[::-1][i])))
encodeCoeffHT=Z1
for i in range(0, len(h.coeffs)):
    encodeCoeffHT=add(encodeCoeffHT, X_HT[i])
print("[HT]1=",encodeCoeffHT)

# CALCULE DE encodeCoeffU + encodeCoeffV
C=add(encodeCoeffHT,encodeCoeffW)
print("[C]1=",C)

#Test de pairing
LPairing = pairing(A, B)
RPairing = pairing(G2, C)

print("\nTest du pairing p(A,B)==p(G2,C) vaut : ",LPairing == RPairing)


======== Affichage de nos polynomes ========
u =  14774563938491510775016323878048660684770145970280823181996287825938670734546x^5 + 3648040478639879203707734290876212514758060733402672390616367364429301415868x^4 + 10032111316259667810196269299909584415584667016857349074195010252180578894213x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602830837x^2 + 8025689053007734248157015439927667532467733613485879259356008201744463116328x + 21888242871839275222246405745257275088548364400416034343698204186575808495064
v =  20429026680383323540763312028906790082645140107054965387451657240804087929235x^5 + 14592161914559516814830937163504850059032242933610689562465469457717205663881x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602830952x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602834704x^2 + 16051378106015468496314030879855335064935467226971758518712016403488926226275x + 1809
w =  11126523459851631571308589587172448

TypeError: Expected an int or FQ object, but got object of type <class 'py_ecc.fields.bn128_FQ2'>